# AES Encryption

Let's try to understand the fundamentals behind AES, and why it is the standard for encryption. The goal is to intuitively understand why it was chosen, and why it works, not just that it happens to work!

Questions:
- Why this specific algorithm, what other algorithms had weaknesses (ie xor)?

--------------------

## Cryptography

Let's try to understand some key cryptographic principles. 

Say we are doing a bitwise transformation, where our key ```010``` either transforms the data (if 0, or keeps it the same, if 1).

```
000
010
|||
101
```

We take our 'plain text' and with a 'key' turn it into 'cipher text' (that can be reversed with the key). So, would an attacker with acess to the cipher be able to derive the plaintext?

Well, if our key *exactly matches* the length of of what we're encrypting, the attacker has no way to efficiently guess the key.

This is because:
- No parts of the key are repeating, its completely random and would need to be derived from scratch
- The crypto text has absolutely *zero* information, if the cypher is random, so is the encrypted text. (equal chance of zero and one for each)

> So, why does using the key multiple times (ie reusing it for 'chunks') reveal information

Well, think of what an attacker can do now.

```
000101010100110110
010--------------- (repeats)
||||||||||||||||||
101...
```

even if the key is very long (not like our three-letter key), the attacker can break it into chunks, and try out different things, for example, trying to construct 'and' and 'the' as many times as possible with a chunk of 24 bits.

The more times the key is repeated, the more words they'll be able to construct, and if the key repeats a *huge* number of times:
- the attacker is able to construct small words with 24 bits (2^24 options only), and the key which makes common words repeat the most is correct
- repeat this with parts of the key, since we can break apart parts of the key.

## Looking beyond Xor

So, how can we use one key, efficiently, to encode our text?

- Avoid being able to split key into parts: a change in plaintext should result in drastic change of key (change ~50% randomly), and avoid patterns in crypto-text (random)

- Avoid being able to *derive the key* from plain and cipher text. Make sure multiple parts of key affect each bit of cipher, and make sure function is non-linear/not solvable.

Basically, it means that you cant reverse engineer the key with math, or split it up, so you'll HAVE to directly guess the key before any pattern shows (in getting the key, or the plaintext). So how is this actually implemented in practice?

## AES (Rinjdael)

After a competition by the NSA, they selected the Rinjdael algorithm, for being optimized for hardware, and for being mathematically/cryptographically sound. But, why?

Now, let's look at the AES algorithm, and see why:
- its secure (anti-frequency, anti-reversibility)
- its fast, and efficient enough to be chosen as the standard


--------------

Todo: 
- [X] Understand why sbox is such; can we do better? --> Derivation?
- [] understand methods of crypto mix and why this is like 'mixing paint' (ie basically random)
- [] understand reason for matrix in mixcolls; and how its inverted
- [] 


Optimization:
- Lookup tables for multiplication GF
- combined?

-----------

# Intuition Behind Gallois Field Arithmetic

When doing arithmetic on our matrix, we want to be able to 'mix' the collumns as much as possible.

The shift rows step is intuitive, however the methods of multiplication, matrix inversion, and addition (xor) in the MixCollumns and InvMixCollumns step may feel arbitrary, at least to someone unfamiliar with gallois field arithmetic. Here is my attempt to discover this in a natural way.

<br>

> ## Representation of Bytes

This is just a review of how we represent numbers when we have bits, or zeros and ones (two possible options).

We can represent numbers of up to 2^n with n bits:
0-255 === {00000000,00000001...}

We could also explore this if we had 3 possible bit options:
0-81 === {0000,0001,0002...2222} --> cover 3^4 possible numbers

the reason this works, is because with coefficients of 1 and 2, we can express 3x1, 3x2 and *then* we move on to 3x3, 3x4 (3^2 term). So we can see how this covers all possible numbers, and extends to any number.

<br>

> ## Polynomial Representation, Fields

Looking at numbers this way is actually uniquely powerful, it allows us to construct a 'field' in which we can do multiplication, and addition (they're reversible), manipulating the structure of numbers in a meaningful (nonlinear) way, that is easily reversible. This is what we need for our encryption!

-----------

# Key Expansion

We also need to make sure that our keys for each round are unique to increase the robustness of the security of our algorithm.

Basically this boils down to mixing our data more, and making the changes less local, and more random per-block (by using multiply rounds, each with a different key).